## Protocol Steps
1. Alice and Bob create n EPR pairs.
2. For each pair, Alice and Bob measure randomly from the {a1,a2,a3} and {b1,b2,b3} basis respectivly. Each of these bases is a rotation around the bloch sphere where a2=b1 and a3=b2
3. Alice and Bob share their measurement bases over a public channel 
4. Alice and Bob both remoove all the bits in their strings corresponding to 0s in the string bob just sent


In [19]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd
from math import pi, sqrt

### Set meta variables

In [44]:
n = 15
test = True
rand_seed = False

if rand_seed:
    np.random.seed(seed=0)

### Step 1: Create n EPR pairs

In [45]:
def create_EPR_pairs(n):
    pairs = []
    for i in range(n):
        qc = QuantumCircuit(2,2)
        qc.h(0)
        qc.cx(0,1)
        pairs.append(qc)
    return pairs

### Step 2: Bob and Alice measure in their repsective bases

In [53]:
# Measures the two bits of an EPR pair with a basis that is the z basis rotated by the given angle
def measure_ang(qca, a_base, b_base):
    # First measure alices bit and then measure bobs

    if a_base == 0:
        # Measure directly in the z basis
        qca.measure(0,0)

    elif a_base == 1:
        # rotate the y basis by pi/4
        qca.u3(pi/4,0,0,0)
        qca.measure(0,0)
        qca.u3(-pi/4,0,0,0)
        
    elif a_base == 2:
        # rotate the y basis by pi/2
        qca.u3(pi/2,0,0,0)
        qca.measure(0,0)
        qca.u3(-pi/2,0,0,0)
        
    if b_base == 0:
        # rotate the z basis by pi/4
        qca.u3(pi/4,0,0,1)
        qca.measure(1,1)
        qca.u3(-pi/4,0,0,1)
        
    elif b_base == 1:
        # rotate the z basis by pi/2
        qca.u3(pi/2,0,0,1)
        qca.measure(1,1)
        qca.u3(-pi/2,0,0,1)
        
    elif b_base == 2:
        # rotate the z basis by 3pi/4
        qca.u3(3*pi/4,0,0,1)
        qca.measure(1,1)
        qca.u3(-3*pi/4,0,0,1)
    
    # Simulate the measurement (taken directly from the qiskit example)
    qasm_sim = Aer.get_backend('qasm_simulator')
    qobj = assemble(qca, shots=1, memory=True)
    result = list(qasm_sim.run(qobj).result().get_memory()[0])    
    return int(result[0]),int(result[1])
    

def measure(pairs, alice_bases, bob_bases):
    alice_outcome, bob_outcome = [],[]
    for i, pair in enumerate(pairs):
        # If measuring in compatible bases, measure and if not add a dummy value of 2
        if (alice_bases[i] == 1 and bob_bases[i] == 0) or (alice_bases[i] == 2 and bob_bases[i] == 1):
            a_out, b_out = measure_ang(pair, alice_bases[i], bob_bases[i])
            alice_outcome.append(a_out)
            bob_outcome.append(b_out)
        else:
            alice_outcome.append(2)
            bob_outcome.append(2)
    return alice_outcome, bob_outcome
        

In [54]:
def trash(a,b,a_bases, b_bases):
    final_key = []
    for i in range(len(a_bases)):
        if (a_bases[i] == 1 and b_bases[i] == 0) or (a_bases[i] == 2 and b_bases[i] == 1):
            final_key.append(a[i])
    return final_key

In [ ]:
def sum_cc():

In [55]:
alice_bases = randint(3, size=n)
bob_bases = randint(3, size=n)


EPR_pairs = create_EPR_pairs(n)

a,b = measure(EPR_pairs, alice_bases, bob_bases)

final = trash(a,b,alice_bases,bob_bases)


[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2]
[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2]
[0 2 2 1 2 2 2 1 1 0 2 0 2 0 0]
[0 2 1 1 2 2 0 1 1 0 1 1 0 1 0]
[1, 0]
